In [1]:
import sys
sys.path.append('..')

In [2]:
from datetime import datetime as dt

import numpy as np
import pandas as pd

from preprocessing import Preprocessing
from sklearn.model_selection import StratifiedKFold, GridSearchCV


from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [3]:
col_drop = ['CustomerId','Surname']
types = {
    'HasCrCard':bool,
    'IsActiveMember':bool
}
train = pd.read_csv('../data/train.csv', index_col='id', dtype=types).drop(columns=col_drop)
train.drop_duplicates(inplace=True)

In [4]:
X = train.drop(columns='Exited')
y = train.Exited

In [5]:
prepro = Preprocessing()
X_scld = prepro.fit_transform(X).astype(float)

In [6]:
skf = StratifiedKFold(n_splits=4)

In [47]:
params = {
    # "boosting_type" : ["gbdt", "dart"],
    "num_leaves" : range(12,19),
    "max_depth" : range(2, 9),
    "n_estimators" : [200, 250, 275],
    # "importance_type" : ["gain", "split"],
}

grid = GridSearchCV(
    LGBMClassifier(),
    params,
    cv=skf,
    scoring='roc_auc',
    n_jobs = -1,
    verbose=10
)

In [48]:
grid.fit(X_scld,y)

Fitting 4 folds for each of 147 candidates, totalling 588 fits
[LightGBM] [Info] Number of positive: 34909, number of negative: 130002
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001289 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 861
[LightGBM] [Info] Number of data points in the train set: 164911, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.211684 -> initscore=-1.314805
[LightGBM] [Info] Start training from score -1.314805
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Li

GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
             estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(2, 9),
                         'n_estimators': [200, 250, 275],
                         'num_leaves': range(12, 19)},
             scoring='roc_auc', verbose=10)

In [49]:
grid.best_params_

{'max_depth': 4, 'n_estimators': 250, 'num_leaves': 16}

In [50]:
grid.best_score_

0.8895195493805318

In [51]:
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,param_num_leaves,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
53,5.986154,0.148851,0.117254,0.005624,4,250,16,"{'max_depth': 4, 'n_estimators': 250, 'num_lea...",0.891259,0.888667,0.889465,0.888687,0.889520,0.001054,1
54,5.866984,0.114777,0.118123,0.004122,4,250,17,"{'max_depth': 4, 'n_estimators': 250, 'num_lea...",0.891259,0.888667,0.889465,0.888687,0.889520,0.001054,1
55,5.915106,0.090489,0.112252,0.002074,4,250,18,"{'max_depth': 4, 'n_estimators': 250, 'num_lea...",0.891259,0.888667,0.889465,0.888687,0.889520,0.001054,1
62,6.090235,0.113297,0.165143,0.033974,4,275,18,"{'max_depth': 4, 'n_estimators': 275, 'num_lea...",0.891228,0.888624,0.889457,0.888654,0.889491,0.001057,4
61,5.870297,0.195301,0.121595,0.001138,4,275,17,"{'max_depth': 4, 'n_estimators': 275, 'num_lea...",0.891228,0.888624,0.889457,0.888654,0.889491,0.001057,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2.518559,0.180633,0.070580,0.008961,2,200,16,"{'max_depth': 2, 'n_estimators': 200, 'num_lea...",0.887924,0.886022,0.884540,0.885434,0.885980,0.001240,141
3,2.355832,0.138329,0.067226,0.001837,2,200,15,"{'max_depth': 2, 'n_estimators': 200, 'num_lea...",0.887924,0.886022,0.884540,0.885434,0.885980,0.001240,141
2,2.325506,0.040276,0.063971,0.004112,2,200,14,"{'max_depth': 2, 'n_estimators': 200, 'num_lea...",0.887924,0.886022,0.884540,0.885434,0.885980,0.001240,141
1,2.074428,0.253270,0.074442,0.016130,2,200,13,"{'max_depth': 2, 'n_estimators': 200, 'num_lea...",0.887924,0.886022,0.884540,0.885434,0.885980,0.001240,141
